In [2]:
# coding: utf-8

# In[3]:


from __future__ import print_function

import sys
import numpy as np
from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))

    print("Start of \"" + sys.argv[0] + "\"")

    ol = Overlay("./design_1.bit")
    ipEx = ol.example_0
    ipDMAIn0 = ol.axi_dma_in_0
    ipDMAIn1 = ol.axi_dma_in_1
    ipDMAOut0 = ol.axi_dma_out_0

    inBuffer0 = allocate(shape=(10,), dtype=np.int32)
    inBuffer1 = allocate(shape=(10,), dtype=np.int32)
    outBuffer0 = allocate(shape=(1,), dtype=np.int32)
    
 
    a=[1,4,9,8,2,5,1,1,5,7]
    b=[7,0,4,8,4,5,7,1,2,6]
    c = [a[b[i]] for i in range (10) if b[i]!=0]
    print('A = '+ str(a))
    print('B = '+ str(b))
    print('\n')
    print('software result')
    print('accumulated_data = ' + str(c))
    print('ans = ' + str(np.sum(c)))
    print('\n')
    
    
    for i in range(10):
        inBuffer0[i]=a[i]
        inBuffer1[i]=b[i]
    
    
    print('FPGA result')
    timeKernelStart = time()

    ipEx.write(0x00, 0x01)
    ipDMAIn0.sendchannel.transfer(inBuffer0)
    ipDMAIn1.sendchannel.transfer(inBuffer1)
    ipDMAOut0.recvchannel.transfer(outBuffer0)
    ipDMAIn0.sendchannel.wait()
    ipDMAIn1.sendchannel.wait()
    ipDMAOut0.recvchannel.wait()
    timeKernelEnd = time()
    print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
    print('output = ' + str(outBuffer0[0]))
    
    
    print("============================")
    print("Exit process")

Entry: /usr/lib/python3/dist-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/lib/python3/dist-packages/ipykernel_launcher.py"
A = [1, 4, 9, 8, 2, 5, 1, 1, 5, 7]
B = [7, 0, 4, 8, 4, 5, 7, 1, 2, 6]


software result
accumulated_data = [1, 2, 5, 2, 5, 1, 4, 9, 1]
ans = 30


FPGA result
Kernel execution time: 0.0006575584411621094 s
output = 30
Exit process
